In [1]:
!pip install transformers datasets accelerate evaluate peft rapidfuzz --quiet
!pip install --upgrade transformers peft accelerate bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.1 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This be

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM ,T5Tokenizer, T5ForConditionalGeneration
from peft import PeftModel
from datasets import load_dataset
from fuzzywuzzy import fuzz
import torch

2025-05-04 07:25:45.869365: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746343546.051497      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746343546.104060      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
model_name = 'google/flan-t5-large'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
from datasets import load_dataset

# Load the first 1000 examples from the training split
squad = load_dataset("squad", split="train[:1000]")

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [5]:
squad

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 1000
})

In [6]:
from datasets import Dataset

train_data = [
    {
        "context": item["context"],
        "question": item["question"],
        "answers": item["answers"]
    }
    for item in squad
]
train_dataset = Dataset.from_list(train_data)
train_dataset


Dataset({
    features: ['context', 'question', 'answers'],
    num_rows: 1000
})

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

total_score = 0
count = 0

for example in squad:
    context = example["context"]
    question = example["question"]
    reference = example["answers"]["text"][0]

    input_text = f"question: {question} context: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True).to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=64)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

    score = fuzz.token_sort_ratio(prediction, reference)
    total_score += score
    count += 1

average_score = total_score / count
print(f"\n🔍 Average Fuzzy Score: {average_score:.2f}")


🔍 Average Fuzzy Score: 92.70


# With Lora 

In [8]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q", "v"],  # Adapted for T5
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 2,359,296 || all params: 785,509,376 || trainable%: 0.3004


In [9]:
def preprocess(example):
    return {
        "input": f"Context: {example['context']}\nQuestion: {example['question']}",
        "output": example["answers"]["text"][0] if example["answers"]["text"] else ""
    }

formatted_dataset = squad.map(preprocess)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
def tokenize(batch):
    inputs = tokenizer(batch["input"], padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(batch["output"], padding="max_length", truncation=True, max_length=128)

    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = formatted_dataset.map(tokenize, batched=True, remove_columns=["input", "output"])


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq
)

training_args = Seq2SeqTrainingArguments(
    output_dir="lora-output",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_dir="logs",
    save_strategy="no",
    fp16=False,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
)

trainer.train()

/tmp/ipykernel_31/2741126461.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,12.601300
1000,0.047900
1500,0.011900
2000,0.012300
2500,0.008900
3000,0.008300


TrainOutput(global_step=3000, training_loss=2.115082941691081, metrics={'train_runtime': 885.439, 'train_samples_per_second': 3.388, 'train_steps_per_second': 3.388, 'total_flos': 6936047124480000.0, 'train_loss': 2.115082941691081, 'epoch': 3.0})

In [ ]:
output_dir = "./output"
trainer.save_model(output_dir)

In [16]:
base_model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)
model = PeftModel.from_pretrained(base_model, "./output")

In [24]:
squad = load_dataset("squad", split="validation[:10]")

test_data = []
for ex in squad:
    input_text = f"Context: {ex['context']}\nQuestion: {ex['question']}"
    ground_truth = ex["answers"]["text"][0] if ex["answers"]["text"] else ""
    test_data.append((input_text, ground_truth))
squad.shape

(10, 5)

In [25]:
def get_prediction(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(model.device)
    output_ids = model.generate(**inputs, max_length=50)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

total_score = 0

for input_text, ground_truth in test_data:
    prediction = get_prediction(input_text)
    score = fuzz.token_sort_ratio(prediction, ground_truth)
    total_score += score

avg_score = total_score / len(test_data)
print(f"\n✅ Average Fuzzy Match Score: {avg_score:.2f}%")



✅ Average Fuzzy Match Score: 96.30%


# With QLora

In [35]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForSeq2SeqLM,BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    "google/flan-t5-large",
    quantization_config=bnb_config,
)
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q", "v", "wi", "wo"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)
model = get_peft_model(model, lora_config)

In [36]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq
)

training_args = Seq2SeqTrainingArguments(
    output_dir="./qlora-output",
    per_device_train_batch_size=1,
    num_train_epochs=2,
    learning_rate=2e-4,
    logging_dir="logs",
    save_strategy="no",
    fp16=False,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
)

# Train model
trainer.train()
# trainer.save_model()

/tmp/ipykernel_31/931716462.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.

Step,Training Loss
500,0.000000
1000,0.000000
1500,0.000000
2000,0.000000


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=2000, training_loss=0.0, metrics={'train_runtime': 1392.9554, 'train_samples_per_second': 1.436, 'train_steps_per_second': 1.436, 'total_flos': 4633091112960000.0, 'train_loss': 0.0, 'epoch': 2.0})

In [37]:
output_dir = "./qlora-output"
trainer.save_model(output_dir)


✅ Average Fuzzy Match Score: 96.30%


In [38]:
base_model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)
model = PeftModel.from_pretrained(base_model, "./qlora-output")
squad = load_dataset("squad", split="validation[:1000]")

In [39]:
test_data = []
for ex in squad:
    input_text = f"Context: {ex['context']}\nQuestion: {ex['question']}"
    ground_truth = ex["answers"]["text"][0] if ex["answers"]["text"] else ""
    test_data.append((input_text, ground_truth))

def get_prediction(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(model.device)
    output_ids = model.generate(**inputs, max_length=50)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [ ]:
total_score = 0

for input_text, ground_truth in test_data:
    prediction = get_prediction(input_text)
    score = fuzz.token_sort_ratio(prediction, ground_truth)
    total_score += score

avg_score = total_score / len(test_data)
print(f"\n✅ Average Fuzzy Match Score: {avg_score:.2f}%")